## 아이템 스카우트 크롤링

In [66]:
%%writefile crawling.py

from urllib.request import *
from bs4 import BeautifulSoup
import ssl
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from openpyxl import Workbook
import pandas as pd
import time

ssl._create_default_https_context = ssl._create_unverified_context

olive = pd.read_excel(r".\원본_올리브영랭크100위화장품정보_상세페이지.xlsx")

#브랜드 이름 추출하는 함수
def product_name(file):
  name_list = [] #브랜드 이름을 담을 리스트
  for i in range(len(file)):
    product_name = file['브랜드'][i] #리스트에 브랜드 저장
    name_list.append(product_name) 
    name_list = list(set(name_list))

  return name_list


#브랜드에 대한 정보들을 알아보는 함수(트렌드 함수)
def trend_index(product): #각 브랜드 이름 인자로 받기
    
    #첫페이지 열기
    driver = webdriver.Chrome('.\chromedriver.exe')
    driver.implicitly_wait(10) # seconds
    driver.get("https://www.itemscout.io/keyword")
    driver.implicitly_wait(10) # seconds
    time.sleep(2) #로딩이 느릴때 쓰기
    
    #키워드 입력, Enter
    keyword = driver.find_element_by_class_name("input-keyword")
    keyword.send_keys(product) #브랜드 이름 입력해서 엔터

    enter = '//*[@id="app"]/div/main/div/div/div/div[1]/div[1]/div[1]/div[2]/div[1]/div'
    enter_click = driver.find_element_by_xpath(enter).click()

    driver.implicitly_wait(10) # seconds
    time.sleep(0.5)

    #한 달 검색수
    pre_elem = driver.find_elements_by_class_name("count-stat")[1]
    search_n = pre_elem.text
    search_n = search_n.replace(',','')

    #Top40 6개월 매출
    pre_elem = driver.find_elements_by_class_name("count-stat")[2]
    sales = pre_elem.text
    sales = sales.replace(",","")
    
    #Top40 6개월 판매량
    pre_elem = driver.find_elements_by_class_name("count-stat")[3]
    volume = pre_elem.text
    volume = volume.replace(",","")

    #Top40 평균 가격
    pre_elem = driver.find_elements_by_class_name("count-stat")[4]
    price = pre_elem.text
    price = price.replace(",","")
    
    driver.close() #다 추출하고 나면 창 닫기
    
    return search_n, sales, volume, price

def process(trend): #전처리 함수
    
    search_n = []
    for i in trend['한 달 검색수']:
        i = int(i.replace("회",""))
        search_n.append(i)
        
    sales = []
    for i in trend['6개월 매출']:
        if i[:-2] == '만원' : #만원단위는 0네개 붙이고 int
            i = i[:-2]
            i=i+'0000'
        else:
            i = i[:-2]
            i=i+'00000000' #억 단위는 0여덟개 붙이고 int
        sales.append(int(i))
        
    sales_volume = []
    for i in trend['6개월 판매량']:
        i = int(i.replace("개",""))
        sales_volume.append(i)
        
    price = []
    for i in trend['평균 가격']:
        i = int(i.replace("원",""))
        price.append(i)
        
    trend['한 달 검색수'] = search_n
    trend['6개월 매출'] = sales
    trend['6개월 판매량'] = sales_volume
    trend['평균 가격'] = price
    
    

if __name__ == '__main__':
    n_list = product_name(olive) #olive파일의 브랜드 이름이 담긴 리스트
    wb = Workbook()
    ws1 = wb.active
    ws1.title = "Trend Index"
    ws1.append(["브랜드", "한 달 검색수", "6개월 매출", "6개월 판매량", "평균 가격"])

    for i in n_list: #겹치지 않는 브랜드 명을 담은 리스트를 인자로 받음
        search_n, sales, volume, price = trend_index(i) #함수 호출
        ws1.append([i, search_n, sales, volume, price]) #정보 저장

    wb.save(filename='new_트렌드지수 크롤링(브랜드).xlsx') #끝나고 나면 엑셀로 만들기
    
    trend = pd.read_excel(r".\new_트렌드지수 크롤링(브랜드).xlsx")
    process(trend) #전처리하여 사용할 수 있게 데이터 준비

Overwriting crawling.py


In [64]:
%run crawling.py

In [65]:
trend

,브랜드,한 달 검색수,6개월 매출,6개월 판매량,평균 가격
0,셀리맥스,15280,2351300000000,16340,15800
1,라운드어라운드,14500,240000000000,2054,13800
2,라네즈,33170,5474400000000,24587,18000


In [50]:
trend['6개월 매출'][0][:-2] + '0000'

'235130000'

In [47]:
sales = []
for i in trend['6개월 매출']:
    if i[:-2] == '만원' : #만원단위는 0네개 붙이고 int
        i = i[:-2]
        print(i)
    else:
        i = i[:-2]
        print(i)
    sales.append(i)
int(sales[0])

23513
2400
54744


23513

## 검색량(monthly) 과 판매량(half-yearly)의 회귀분석

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

Y = trend['6개월 판매량'].values
Y = Y.reshape(-1,1)

X = trend['한 달 검색수'].values
X = X.reshape(-1,1)

lr.fit(X,Y)

print(lr.coef_[0])  #검색량이 판매량에 미치는 상관계수가 '0.49'이 나옴 (검색량이 하나 늘어날때마다 0.49상품 만큼의 판매량이 증가한다.)
print(lr.intercept_)

In [ ]:
plt.scatter(X,Y)
#plt.plot(X,Y2,color='red')
plt.title('Y = {}*X + intercept'.format(lr.coef_[0], lr.intercept_))
plt.xlabel('Montly Search number')
plt.ylabel('Sales volumer for half year')
plt.show

In [ ]:
import statsmodels.api as sm
results = sm.OLS(Y, sm.add_constant(X)).fit()

results.summary() #12%의 설명력을 가진다. P-value또한 0.05보다 작음으로 결과 값은 유의미하다.

In [ ]:
P = trend['평균 가격'].values
P = P.reshape(-1,1)

S = trend['6개월 매출'].values
S = S.reshape(-1,1)

fig = plt.figure(figsize=(10, 8))

plt.subplot(221)
plt.scatter(X,S,alpha=0.5)
plt.title('Search numbers and Sales volume')
plt.xlabel('Search numbers')
plt.ylabel('Sales')

plt.subplot(222)
plt.scatter(P,Y,alpha=0.5)
plt.title('Price and Sales_Volume')
plt.xlabel('Price')
plt.ylabel('Sales volume')

plt.subplot(223)
plt.scatter(S,Y,alpha=0.5)
plt.title('Sales and Sales volume')
plt.xlabel('Sales')
plt.ylabel('Sales volume')

plt.subplots_adjust(wspace = 0.5, hspace = 0.6)

plt.show

In [57]:
n_trend = pd.read_excel(r'.\new_트렌드지수 크롤링(브랜드).xlsx')

In [58]:
n_trend

,Unnamed: 0,브랜드,한 달 검색수,6개월 매출,6개월 판매량,평균 가격
0,1,구달,13130,13119,8261,18600
1,2,한율,25200,30873,10223,35100
2,3,에스네이처,4950,5269,2118,23100
3,4,에스티로더,78110,110000,9762,104600
4,5,코스알엑스,17860,38557,24982,14700
5,6,나인위시스,13620,20953,8566,21400
6,7,아벤느,12440,48188,22975,19400
7,8,클린업뷰티,2060,571,583,57900
8,9,닥터지,55900,84015,48954,16300
9,10,라운드랩,15240,110000,63247,17000
